# Modelo B 


por: Catalina Alvarez 

# 1. Preparación del Ambiente

In [ ]:
import pandas as pd
#from ydata_profiling import ProfileReport

from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import SnowballStemmer, WordNetLemmatizer
from sklearn.preprocessing import FunctionTransformer
from sklearn.inspection import permutation_importance
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from nltk.tokenize import word_tokenize
from sklearn.pipeline import Pipeline
from wordcloud import WordCloud
#from num2words import num2words
import matplotlib.pyplot as plt
import re, string, unicodedata
import matplotlib

%matplotlib inline
import numpy as np
import inflect
import joblib
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
from nltk.tokenize import word_tokenize

ModuleNotFoundError: No module named 'wordcloud'

# Carga de datos

In [ ]:
df_original = pd.read_csv(r'datos\fake_news_spanish.csv', sep=';', encoding='utf-8')
df_original.head()


In [ ]:
train = df_original.copy()
train.head()

# 2. Calidad de los datos

Se van a analizar 4 dimensiones distintas con respecto a la calidad de los datos (Completitud, Validez, Unicidad y Consistencia)

# Completitud

Bajo una inspeccion sencilla, parece no haber problemas de completitud en los datos

In [ ]:
train.isnull().sum()

1    33158
0    23905
Name: Label, dtype: int64

In [ ]:
train.count()

0

In [ ]:
train['Label'].value_counts()

In [ ]:
# strip all values in the column
train['Descripcion'] = train['Descripcion'].str.strip()
# count all values in the column where the value is an empty string
train['Descripcion'].isnull().sum()

# Validez 
Se identifican los siguientes problemas de validez: 
Los textos en el dataset tienen problemas, vocales con tilde han sido remplazados por caracteres no validos. 


In [ ]:
def fix_encoding(text):
    replacements = {
        'Ã¡': 'á',
        'Ã©': 'é',
        'Ã­': 'í',
        'Ã³': 'ó',
        'Ãº': 'ú',
        'Ã±': 'ñ'
    }
    
    for wrong, correct in replacements.items():
        text = text.replace(wrong, correct)
    
    return text

train['Descripcion'] = train['Descripcion'].apply(fix_encoding)

Convertir la columna Titulo de float a string para aplicar los cambios de fix_encoding

In [ ]:
# Convertir la columna "Titulo" de float a string
train["Titulo"] = train["Titulo"].astype(str)

# Verificar el cambio de tipo de dato
train.dtypes["Titulo"]

,ID,Label,Titulo,Descripcion,Fecha
0,ID,1,'The Guardian' va con Sánchez: 'Europa necesit...,El diario británico publicó este pasado jueves...,02/06/2023
1,ID,0,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,01/10/2023
2,ID,1,El 'Ahora o nunca' de Joan Fuster sobre el est...,El valencianismo convoca en Castelló su fiesta...,25/04/2022
3,ID,1,"Iglesias alienta a Yolanda Díaz, ERC y EH Bild...","En política, igual que hay que negociar con lo...",03/01/2022
4,ID,0,Puigdemont: 'No sería ninguna tragedia una rep...,"En una entrevista en El Punt Avui, el líder de...",09/03/2018


In [ ]:
train['Titulo'] = train['Titulo'].apply(fix_encoding)

In [ ]:
train.head(5)

# Unicidad 
En primera instancia, el dataset no tiene columnas unicas sigueindo alguna restriccion de negocio. Luego de una sencilla inspeccion, parece haber filas totalmente duplicadas. 

In [ ]:
train.duplicated().sum()

445

# Consistencia 
La coluumna "ID" no es unica en su totalidad, es posible que haya inconsistencias 

In [ ]:
train["ID"].nunique() == len(train)

# Perfilamiento de datos 

Eliminacion de datos duplicados

In [ ]:
train.drop_duplicates(inplace=True)

In [ ]:
train.count()

In [ ]:
def mostrar_grafico1(palabras,stopwords=[]):
    comment_words = ''
    
    # iterate through the csv file
    for val in palabras:
        
        # typecaste each val to string
        val = str(val)
    
        # split the value
        tokens = val.split()
        
        # Converts each token into lowercase
        for i in range(len(tokens)):
            tokens[i] = tokens[i].lower()
        
        comment_words += " ".join(tokens)+" "
    
    
    wordcloud = WordCloud(width = 600, height = 600,
                    background_color ='white',
                    stopwords = stopwords,
                    min_font_size = 10).generate(comment_words)
    
    # plot the WordCloud image                      
    plt.figure(figsize = (8, 8), facecolor = None)
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")
    plt.tight_layout(pad = 0)
    
    plt.show()

A continuacion, se ilustrara que palabras suelen repetirse y encontrarse en los Label's 0 y 1 

In [ ]:
stop_words = set(nltk.corpus.stopwords.words('spanish'))

for i in train['Label'].unique():
    print(f'---------- Words for class: {i} ----------')
    mostrar_grafico1(train.loc[train['Label']==i,'Titulo'], stopwords=stop_words)

In [ ]:
train

# Preparacion de datos

In [ ]:
X_train, y_train = train['Titulo'], train['Label'] 
display(X_train)
y_train

# Modificacion de Caracteres

Se pasan todos los caracteres a minusculas, se eliminan textos de puntuacion y stop words (suelen ser articulos y conectores)

1. Cambiar minusculas 

In [ ]:
def to_lower(words: list[str]):
    return [word.lower() for word in words]

2. Quitar puntuacion

In [ ]:
def remove_punctuation(words: list[str]):
    return [word for word in words if word.isalnum()]

3. Eliminar StopWords

In [ ]:
def remove_stopwords(words: list[str]):
    stopwords = set(nltk.corpus.stopwords.words('spanish'))
    return [word for word in words if word not in stopwords]

In [ ]:
def preprocessing(words):
    words = to_lower(words)
    words = remove_punctuation(words)
    words = remove_stopwords(words)
    return words

Ahora se gace un procedimiento de Tokenizacion para desglozar palabras 

In [ ]:
new_X_train = X_train.apply(word_tokenize)
new_X_train = new_X_train.apply(preprocessing) 
new_X_train.head()

In [ ]:
def stem_words(words):
    """Stem words in list of tokenized words"""
    stemmer = SnowballStemmer('spanish')
    stems = []
    for word in words:
        stem = stemmer.stem(word)
        stems.append(stem)
    return stems

In [ ]:

def lemmatize_verbs(words):
    """Lemmatize verbs in list of tokenized words"""
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for word in words:
        lemma = lemmatizer.lemmatize(word, pos='v')
        lemmas.append(lemma)
    return lemmas

In [ ]:
def stem_and_lemmatize(words):
    #words = stem_words(words)
    words = lemmatize_verbs(words)
    return words

In [ ]:
new_X_train = new_X_train.apply(stem_and_lemmatize) 
new_X_train.head()